## Skip-gram모델을 활용 Word2Vec실습

In [ ]:
# 불용어(Stopword)를 제거하기 위해 nltk 패키지를 활용
# https://direction-f.tistory.com/29  자료 겅리한 것

In [7]:
import os
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JYB\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
# text8 데이터를 활용했으며, 이는 Kaggle 데이터셋에서 다운받을 수 있습니다. 
# 데이터를 읽어서 각 단어의 빈도를 계산하겠습니다. 
# 여기서는 읽은 데이터 중에서 10000개까지만 읽어서 진행하겠습니다.

# nltk패키지를 이용하여 불용어를 제거하고, 단어가 나올때마다 +1를 해줍니다.

In [9]:
wc_dict={}
step=0      
with open('./text8',"r") as f:
    text = f.read()
    sent = text[:10000].split()
    stop_words = set(stopwords.words("english"))
    sent_filter = [w for w in sent if not w in stop_words]
    for word in sent_filter:
        step+=1
        print(step)
        wc_dict[word] = wc_dict.get(word,0)+1
        
 wc_dict

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

In [ ]:
idx2word = ["<UNK>"]+sorted(wc_dict, key =wc_dict.get, reverse= True)[:max_vocab-1] ## 큰수부터 max_vocab 까지 가지고 옮(여기선 전부)
word2idx = {idx2word[idx]:idx for idx, _ in enumerate(idx2word)}
vocab = set([word for word in word2idx]) ## dict의 ket값만 저장

In [ ]:
# Context데이터를 만들기 위한 함수를 정의하는데, 
# 첫 번째 단어같은 경우는 전에 나온 단어가 없기 때문에 "<UNK>"로 Padding을 해줍니다.

In [ ]:
def skipgram(sentence, i, window_size = 2):
    iword = sentence[i]
    left = sentence[max(i-window_size,0):i] ## i(anchor)보다 전에 나온 단어들
    right = sentence[i+1:i+1+window_size] ## i(anchor)보다 후에 나온 단어들
    
    
    ## Padding 추가
    return iword, ["<UNK>"for _ in range(window_size - len(left))] + left + right + ["<UNK>" for _ in range(window_size - len(right))]

sent =[]
data=[]
step=0
with open('./text8',"r") as f:
    text = f.read()
    words = text[:10000].split()
    stop_words = set(stopwords.words("english"))
    words_filter = [w for w in words if not w in stop_words]
  
    for word in words_filter:
        
        step +=1
        print(step)
        if word in vocab:            
            sent.append(word)
        
        else :
            sent.append("<UNK>")
        
       
    for i in range(len(sent)):
        print(i)
        iword, owords = skipgram(sent, i)
        data.append((word2idx[iword], [word2idx[oword] for oword in owords]))

In [ ]:
# 위의 코드를 실행하게 되면(1, [2,4,5,7])과 같이 구성되게 됩니다. 
# 즉 1번 단어에 앞뒤로 2,4,5,7번의 단어가 나오게 된 것입니다. 이번에 학습할때 우리는 pair 데이터를 활용하려고 합니다. 
# 따라서 (1, [2,4,5,7])를 (1,2), (1,4),(1,5),(1,7)과 같이 pair를 만들어 학습을 진행하고자 합니다.

In [ ]:
def get_batches(data, batch_size=950): ## batch_size는 조정가능
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(data)//batch_size
    
    # only full batches
    words = data[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii][0]
            batch_y = batch[ii][1]
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y)) ## 같은 값을 (window_size)만큼 늘림
        yield x, y 
        
x,y=next(get_batches(data)) ## test

In [ ]:
# 주요 Concept은 관심단어를 Embedding 하고 
# Context 데이터도 Embedding 하여 각 Embedding한 Vector들의 dot product값을 최대화하는 것과
# 동시에 Negative Sampling한 데이터를 Embedding Vector와의 dot product는 최소화 하는 것입니다.
# 위 그램에서 전자가 Context data와의 dot product를 나타내고 후자가 negative sampling한 데이터와의 dot product를 나타냅니다. 각 dot product한 값들에 Sigmoid function과 log를 취합니다. 위 함수를 최대화하도록 학습하게 됩니다. 
# 실제 Pytorch에서 표현할 때는 "-" 를 곱해주어 최소화 문제로 변경합니다.

In [ ]:
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        # define embedding layers for input and output words
        self.in_embed = nn.Embedding(n_vocab,n_embed)
        self.out_embed = nn.Embedding(n_vocab,n_embed)
        
        # Initialize both embedding tables with uniform distribution
        self.in_embed.weight.data.uniform_(-1,1)
        self.out_embed.weight.data.uniform_(-1,1)
        
    def forward_input(self, input_words):
        
        input_vector = self.in_embed(input_words)
        return input_vector
    
    def forward_output(self, output_words):
        
        output_vector = self.out_embed(output_words)

        return output_vector
    
    def forward_noise(self, batch_size, n_samples):
        """ Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        if self.noise_dist is None:
            # Sample words uniformly
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
            
        # Sample words from our noise distribution
        noise_words = torch.multinomial(noise_dist,
                                        batch_size * n_samples,
                                        replacement=True)  ## noise sample 만큼 데이터 생성 
    

        noise_vector = self.out_embed(noise_words).view(batch_size,n_samples,self.n_embed)        
        return noise_vector
        
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_vectors, output_vectors, noise_vectors):
        
        batch_size, embed_size = input_vectors.shape
        
        # Input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # Output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication (b*n*e)(b*e*n) = (b*n*n)
        # correct log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        
        # incorrect log-sigmoid loss
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        ## 각 row별로 loss 합산(e.g. [input-negative1 loss] +[input-negative2 loss]
        noise_loss = noise_loss.squeeze().sum(1)  
        return -(out_loss + noise_loss).mean()

In [ ]:
# Negative Sampling을 하기 위한 Noise Distribution을 생성해줍니다. 
# torch.mutinomial에서 input으로 들어가는 값은 각 index를 반환할 확률이라고 이해하시면 될 것 같습니다. 
# 따라서 Noise Distribution을 생성한다는 것은 어떤 index를 반환할지에 대한 확률값을 주는 것입니다. 

# 확률값을 줄 때, 우리 단어중에서 나온 빈도가 높은 단어가 높은 확률을 가지는 것은 자연스러울 것입니다. 
# 따라서 빈도가 많이 나온 단어들은 상대적으로 높은 확률값을 가지게 됩니다. 
# 우리 데이터 원본에는 "<UNK>"가 없기 때문에 별도로 index 0자리에 생성확률 0으로 추가해주었습니다. 
# negative sample로 뽑힐 확률은 아래와 같이 3/4승을 많이 적용합니다.(실험적으로 성능이 좋다고 합니다.)

In [ ]:
total_count = len(idx2word)
freqs = {word:count/total_count for word, count in wc_dict.items()} 
word_freqs = np.array(sorted(freqs.values(), reverse=True))
word_freqs = np.concatenate(([0],word_freqs)) ## "<UNK>" 추가
unigram_dist = word_freqs/word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

In [ ]:
embedding_dim = 30
model = SkipGramNeg(len(vocab), embedding_dim, noise_dist =noise_dist)

criterion = NegativeSamplingLoss()
optimizer = optim.Adam(model.parameters(), lr =0.03)

for epoch in range(1000):
    for input_words, target_words in get_batches(data):
        step +=1
        inputs, targets = torch.LongTensor(input_words), torch.LongTensor(target_words)
        input_vectors = model.forward_input(inputs)
        output_vectors = model.forward_output(targets)
        noise_vectors = model.forward_noise(inputs.shape[0], 5)
        
        loss = criterion(input_vectors, output_vectors, noise_vectors)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 100 == 0:
            print("Epoch: {}/{}".format(epoch+1, 1000))
            print("Loss: ", loss.item())​

In [ ]:
# Embedding된 벡터를 활용하여, Cosine similarity를 이용해 가장 가까이 있는 단어 검색

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

embeddings = model.in_embed.weight.data.numpy()
def cosine_(data):
    close_idx= np.zeros([len(data),len(data)])
    for i in range(len(data)):
        for j in range(len(data)):
            close_idx[i,j]=cosine_similarity (data[i].reshape(1,-1),data[j].reshape(1,-1))
            
    return close_idx

close_idx=cosine_(test_results)

def closest_word(close_idx,word, word2idx, idx2word):
    idx=word2idx[word]
    close_idx_=close_idx[idx].argsort()[:5]
    close_Word = [idx2word[i] for i in close_idx_]
    return close_Word

for i in vocab:    
    sim_list = closest_word(close_idx, i, word2idx, idx2word)
    
    if "<UNK>" in sim_list:
        sim_list.remove("<UNK>")
    print("word:{} : {}".format(i,sim_list))
    
## Output
##word:twenty : ['spirit', 'holy', 'citium', 'premise']
##word:owners : ['authoritarian', 'communists', 'movements', 'polarised', 'later']
##word:doctrine : ['access', 'communism', 'supported', 'way', 'favoured']
##word:mikhail : ['word', 'institutions', 'anarchy', 'private']
##word:new : ['incorporated', 'used', 'history', 'harsh'

In [ ]:
# TSNE()를 활용해서 시각화를 해보겠습니다. 
# TSNE는 고차원의 벡터를 저차원의 벡터로 차원 축소해주는 알고리즘입니다.
# 시각화

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

viz_words = len(idx2word)
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])
embed_tsne_plot =pd.DataFrame(embed_tsne, columns =["x","y"])

fig, ax = plt.subplots(figsize=(16, 16))
plt.scatter(embed_tsne_plot["x"],embed_tsne_plot["y"], color='steelblue' )
for idx in range(viz_words):
    plt.annotate(idx2word[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)​